In [1]:
import pandas as pd
import os
from pathlib import Path

In [2]:
datapath = Path('/cfs/earth/scratch/kraftjul/DaAn_Projektarbeit/data/')

In [13]:
def path_to_year(year):
    path = datapath / year
    return path

def files_in_year(year, filter=None):
    all_files = os.listdir(path_to_year(year))
    if filter:
        return [f for f in all_files if filter in f]
    else:
        return all_files

In [36]:
def aggregate_data_per_file(file_path, save_file_path: Path):
    data = pd.read_csv(file_path)

    # filtering for line 1 - 20 (only tram) and fw_typ 1 (only regular service)
    filtered_data = data.where((data['linie'] <= 20) & (data['fw_typ'] == 1)).dropna()

    # calculate delay
    filtered_data['verspaetung'] = filtered_data['ist_an_von'] - filtered_data['soll_an_von']

    # select relevant columns
    cols_to_select = ['linie', 'betriebsdatum', 'fahrzeug', 'soll_an_von', 'verspaetung', 'halt_id_von']
    selected_data = filtered_data[cols_to_select]

    # ensure the directory for save_file_path exists
    if not save_file_path.parent.exists():
        save_file_path.parent.mkdir(parents=True)

    with save_file_path.open('a') as f:
        selected_data.to_csv(f, index=False, header=f.tell()==0)


In [40]:
os.listdir(datapath)

['2016', '2018', '2021', '2020', '2022', '2019', '2017']

In [37]:
test_file = Path('/cfs/earth/scratch/kraftjul/DaAn_Projektarbeit/data/2016/Fahrzeiten_SOLL_IST_20160103_20160109.csv')
test_save_path = Path('/cfs/earth/scratch/kraftjul/DaAn_Projektarbeit/data/aggregated/2016_test.csv')

In [39]:
aggregate_data_per_file(test_file, test_save_path)